In [7]:
import json
import itertools
import os

def json_cfg_to_dot(cfg_json, graph_name="MyGraph"):

    # 使用一个生成器来给 subgraph 命名，防止重名
    cluster_counter = itertools.count(1)

    # 用一个内部函数递归地构造 DOT 代码
    def process_code_block(code_block, dot_lines, parent_cluster_name=None):
        """
        处理单个代码块（可能是顶层，也可能是 class、function）。
        - 在 DOT 中创建一个 subgraph。
        - subgraph 里包含本块的所有 basic blocks 作为节点，及其之间的边。
        - 对于嵌套的 classes / functions，递归创建子 subgraph。
        """

        # 如果既没有 classes，又没有 functions，就不用再嵌套子 subgraph，直接画节点即可；
        # 否则，把当前块当成一个 subgraph，把其 blocks 画进去，然后再对嵌套的 classes/functions 各自再建 subgraph。
        block_name = code_block.get("name", "TopLevel")
        block_type = code_block.get("type", "")
        start_line = code_block.get("start_line", 0)
        end_line = code_block.get("end_line", 0)

        # 生成一个子图的名字，比如 cluster_1, cluster_2 ...
        this_cluster_id = next(cluster_counter)
        # 让 subgraph 的名字中含有 “cluster_” 前缀，这样有助于 Graphviz 渲染
        subgraph_name = f"cluster_{this_cluster_id}"

        # subgraph label：可根据需要自定义，这里展示 name + 行号范围
        subgraph_label = f"{block_name} ({block_type}) [L{start_line}-L{end_line}]"

        dot_lines.append(f'  subgraph "{subgraph_name}" {{')
        # 可以加上一个可读的 label
        dot_lines.append(f'    label="{subgraph_label}";')

        # 先画本代码块自己的 basic blocks
        blocks = code_block.get("blocks", [])
        # 为了后续画 edges，需要记住 “block_id -> 节点名” 的对应关系
        block_id_to_node_name = {}

        for b in blocks:
            node_name = f'node_{this_cluster_id}_{b["id"]}'
            block_id_to_node_name[b["id"]] = node_name

            # label 里保留换行，以便在 Graphviz 中多行显示
            # 也可以使用 "\l" 等转义来让 Graphviz 左对齐显示
            label_str = b["label"].strip().replace("\\n", "\\l")  # 先做字符串替换
            node_label = f"id: {b['id']}\\n" + label_str          # 再与 f-string 拼接

            # 也可以把换行替换为 \n，然后用 shape=box, label just="l" 之类的自定义
            dot_lines.append(f'    {node_name} [label="{node_label}", shape=box];')

        # 再画每个 block 的 successors 边
        def draw_edges_recursively(current_block, current_node_name):
            successors = current_block.get("successors", [])
            for succ_block in successors:
                succ_id = succ_block["id"]
                if succ_id in block_id_to_node_name:
                    succ_node_name = block_id_to_node_name[succ_id]
                    dot_lines.append(f'    {current_node_name} -> {succ_node_name};')
                    # 递归处理 successor 的 successors（仅当 successor 不在同一个 blocks 列表时）
                    # 这里的情况一般不会发生嵌套 successors 不在 blocks 里，但以防万一做个判断
                    draw_edges_recursively(succ_block, succ_node_name)
                else:
                    # successor 如果本层 blocks 里没有，可能是更深层次或者异常情况
                    # 也可以自行处理；这里暂时忽略
                    pass

        # 对 blocks 的每个 successor，画边
        for b in blocks:
            node_name = block_id_to_node_name[b["id"]]
            draw_edges_recursively(b, node_name)

        # 然后递归处理嵌套的 classes / functions
        for cls in code_block.get("classes", []):
            process_code_block(cls, dot_lines, parent_cluster_name=subgraph_name)
        for func in code_block.get("functions", []):
            process_code_block(func, dot_lines, parent_cluster_name=subgraph_name)

        # 结束 subgraph
        dot_lines.append(f'  }}')  # subgraph

    # 开始构造 DOT
    dot_lines = []
    dot_lines.append(f'digraph "{graph_name}" {{')

    # 处理最顶层的 code_block（即传进来的整个 cfg_json）
    process_code_block(cfg_json, dot_lines)

    dot_lines.append("}")
    return "\n".join(dot_lines)


# ------------------ 以下为一个简单的演示 ------------------
if __name__ == "__main__":
    llm_cfg_dir = "merged_llm_cfg"
    output_dir = "cfg_dot"
    os.makedirs(output_dir, exist_ok=True)
    for i in range(200):
        if not os.path.exists(f"{llm_cfg_dir}/{i}.json"):
            continue
        cfg_json = json.load(open(f"{llm_cfg_dir}/{i}.json"))
        dot_output = json_cfg_to_dot(cfg_json, graph_name=f"{i}.ts")
        with open(f"{output_dir}/{i}.ts.dot", "w") as f:
            f.write(dot_output)
